## Circadian Rhythm Risk Calculation

In [1]:
import pandas as pd #Pandas is high performance data manipulation 
import numpy as np   #numpy is for array processing
import random as rd
import matplotlib.pyplot as plt   # matplot is for python graphics
import seaborn  as sns
# pd.set_option('max_columns', None)

In [2]:
# Read in the integrated Data Files 
# df_Patient = pd.read_excel("C:/Users/josep/documents/datasets/AIHS_Patient.xlsx",nrows=10000)
df_Patient = pd.read_excel("C:/Users/josep/documents/datasets/AIHS_Patient.xlsx",nrows=1000)
df_Patient.head()

,Index,Year,Patient Ordinal,Birth Date,Age,Decade,Gender,Race,Race Category,Diabetes,...,PAM Q4,PAM Q5,PAM Q6,PAM Q7,PAM Q8,PAM Q9,PAM Q10,PAM Q11,PAM Q12,PAM Q13
0,0,2020,Patient_1,1972-04-12,49,4,Male,1,White,0,...,5,2,3,4,4,1,1,2,5,4
1,1,2020,Patient_2,1947-09-03,73,7,Male,1,White,0,...,2,1,2,4,1,5,4,5,4,5
2,2,2020,Patient_3,1971-09-16,49,4,Male,1,White,0,...,2,1,1,4,5,3,1,1,1,5
3,3,2020,Patient_4,1934-01-08,87,8,Female,1,White,0,...,2,1,5,2,1,4,4,3,4,5
4,4,2020,Patient_5,1934-05-12,87,8,Male,1,White,0,...,3,5,5,5,4,3,1,3,3,5


In [3]:
df_Patient.to_excel('c:\\working_directory\\excel\\AIHS_patient.xlsx')

In [4]:
df_Patient.columns 

Index(['Index ', 'Year', 'Patient Ordinal', 'Birth Date', 'Age', 'Decade',
       'Gender', 'Race', 'Race Category', 'Diabetes',
       ...
       'PAM Q4', 'PAM Q5', 'PAM Q6', 'PAM Q7', 'PAM Q8', 'PAM Q9', 'PAM Q10',
       'PAM Q11', 'PAM Q12', 'PAM Q13'],
      dtype='object', length=491)

In [5]:
df_PERC = df_Patient[[ 'Age','Gender','Diabetes','BMI','Simulated BMI Class','Total Calculated AIHS Risk Stratifcation','PERSDOC2', \
                       'ADEAT1','ALCDAY5','AVEDRNK2','MAXDRNKS','ADANXEV','SMOKE100',  \
                       'SMOKDAY2','STOPSMK2','USENOW3','MEDCOST',]]

In [6]:
df_PERC.rename(columns={"ADANXEV": "Adverse Childhood Event"}, inplace=True)
df_PERC.rename(columns={"ADEAT1": "History of Anxiety and Depression"}, inplace=True)
df_PERC.rename(columns={"ALCDAY5": "Alcohol Days last 30 Days or Week"}, inplace=True)
df_PERC.rename(columns={"SMOKE100": "Have you smoked >100 cigerettes in life"}, inplace=True)
df_PERC.rename(columns={"SMOKDAY2": "Do you now smoke cigarettes every day, some days, or not at all?"}, inplace=True)
df_PERC.rename(columns={"USENOW3": "Do you currently use any tobacco products?"}, inplace=True)
df_PERC.rename(columns={"STOPSMK2": "Have you tried to stop smoking?"}, inplace=True)
df_PERC.rename(columns={"MEDCOST": "Have you avioided necessary medical do to cost in last year?"}, inplace=True)
df_PERC.rename(columns={"ALCDAY5": "During the past 30 days how many days per week or per month did you have at least one drink of any alcoholic"}, inplace=True)
df_PERC.rename(columns={"AVEDRNK2": "During the past 30 days on the days when you drank about how many drinks average"}, inplace=True)
df_PERC.rename(columns={"MAXDRNKS": "During the past 30 days, what is the largest number of drinks you had on any occasion?"}, inplace=True)
df_PERC.rename(columns={"DRNK3GE5": "Any alcoholic beverages, how many times during the past 30 days did you have"}, inplace=True)
df_PERC.rename(columns={"PERSDOC2": "Do you have one person you think of as your personal doctor or health care provider?"}, inplace=True)


C:\Users\josep\AppData\Local\Temp\ipykernel_20200\505942946.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PERC.rename(columns={"ADANXEV": "Adverse Childhood Event"}, inplace=True)
C:\Users\josep\AppData\Local\Temp\ipykernel_20200\505942946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PERC.rename(columns={"ADEAT1": "History of Anxiety and Depression"}, inplace=True)
C:\Users\josep\AppData\Local\Temp\ipykernel_20200\505942946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [7]:
def interpret_acute_childhood_event(row):
    if (row['Do you currently use any tobacco products?'] == 1.0):  
        row['Acute Childhood Event']= 1
    else: row['Acute Childhood Event'] = 0      
    return row

def interpret_anxiety_depression_history(row):
    if (row['History of Anxiety and Depression'] == 1.0):  
        row['History of Anxiety or Depression']= 1
    else: row['History of Anxiety or Depression'] = 0      
    return row

def interpret_financial_stress(row):
    if (row['Have you avioided necessary medical do to cost in last year?'] == 1.0):  
        row['Financial Stress']= 1
    else: row['Financial Stress'] = 0      
    return row

def interpret_binging_overeating(row):
    #  Interpret BMI for Binging, Overeating, emotional Eating
    if (row['BMI'] > 30):  
        row['Binging or Overeating or Emotional Eating']= 1
    else: row['Binging or Overeating or Emotional Eating'] = 0      
    return row

def interpret_sweet_tooth(row):
    #  Interpret BMI for Binging, Overeating, emotional Eating
    if (row['BMI'] > 30):  
        row['Sweet tooth or Sugar Drinks']= 1
    else: row['Sweet tooth or Sugar Drinks'] = 0      
    return row

def interpret_overweight_struggling_to_lose(row):
    #  Interpret BMI for Binging, Overeating, emotional Eating
    if (row['BMI'] > 30):  
        row['Overweight Struggling to Lose']= 1
    else: row['Overweight Struggling to Lose'] = 0      
    return row

def interpret_escape_or_coping_mechanism(row):
    #  Interpret Escape or copying using TV or regular alcohol consumption
    if (row['During the past 30 days, what is the largest number of drinks you had on any occasion?'] > 3.0):   
        row['Escape or Coping Mechanism']= 1
    else: row['Escape or Coping Mechanism'] = 0      
    return row      
              
def interpret_binge_drinking(row): 
    # The row is a single Series object which is a single row indexed by column values
    if (row['During the past 30 days, what is the largest number of drinks you had on any occasion?'] > 3.0):  
        row['Binge Drinking']= 1
    else: row['Binge Drinking'] = 0      
    return row              
    
def interpret_smoking_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    if row['Do you currently use any tobacco products?'] == 1.0: 
        row['Smoker']= 1
    else: row['Smoker'] = 0      
    return row              
              
def interpret_stop_smoking_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    if (row['Have you tried to stop smoking?'] == 1.0):  
        row['Failed Attempts to Stop Smoking']= 1
    else: row['Failed Attempts to Stop Smoking'] = 0      
    return row  
              
def interpret_sedentary(row):
    if (row['BMI'] > 30):
        row['Sedentary Lifestyle']= 1
    else: row['Sedentary Lifestyle'] = 0      
    return row      

def interpret_self_efficacy(row):
    if (row['Have you avioided necessary medical do to cost in last year?'] == 1.0):  
        row['Self Efficacy']= 1
    else: row['Self Efficacy'] = 0      
    return row      
              
def interpret_social_support_network(row):
    if (row['Have you avioided necessary medical do to cost in last year?'] == 1.0):  
        row['Poor Social Support']= 1
    else: row['Poor Social Support'] = 0      
    return row      
                 
              
def sum_PERC_risk_Factors(row):
    row['Poor Emotional Regulatory Control Factors'] =  row['Acute Childhood Event'] + row['History of Anxiety or Depression'] + \
    row['Financial Stress'] + row['Binging or Overeating or Emotional Eating'] + \
    row['Sweet tooth or Sugar Drinks'] + row['Overweight Struggling to Lose'] + \
    row['Escape or Coping Mechanism'] + row['Smoker'] + \
    row['Failed Attempts to Stop Smoking']  +  row['Binge Drinking'] + \
    row['Sedentary Lifestyle'] + \
    row['Self Efficacy'] + row['Poor Social Support']
    return row  
              
def sum_PERC_risk_Strat(row):
    row['Poor Emotional Regulatory Control Risk'] =  row['Poor Emotional Regulatory Control Factors'] / 14
    return row
              
def sum_PERC_risk_Score(row):
    row['Poor Emotional Regulatory Control Score'] =  (row['Acute Childhood Event'] * 1.0) + (row['History of Anxiety or Depression'] * 1.0)+ \
    (row['Financial Stress'] * 1.0) + (row['Binging or Overeating or Emotional Eating'] * 1.0) + \
    (row['Sweet tooth or Sugar Drinks'] * 1.0) + (row['Overweight Struggling to Lose'] * 1.0) + \
    (row['Escape or Coping Mechanism'] * 1.0) + (row['Smoker'] * 1.0) + \
    (row['Failed Attempts to Stop Smoking'] * 1.0) +  (row['Binge Drinking'] * 1.0) + \
    (row['Sedentary Lifestyle'] * 1.0) + \
    (row['Self Efficacy'] * 1.0) + (row['Poor Social Support'] * 1.0)
    return row     

def sum_PERC_risk_Cohort(row):
    perc_cohort = '' 
    if (row['Acute Childhood Event'] == 1.0):
        perc_cohort = perc_cohort + 'Acute Childhood Event, '
    if (row['History of Anxiety or Depression'] == 1.0):
        perc_cohort = perc_cohort + 'Anxiety or Depression, '
    if (row['Financial Stress'] == 1.0):
        perc_cohort = perc_cohort + 'Financial Stress, '       
    if (row['Binging or Overeating or Emotional Eating'] == 1.0):
        perc_cohort = perc_cohort + 'Binge Eating, ' 
    if (row['Sweet tooth or Sugar Drinks'] == 1.0):
        perc_cohort = perc_cohort + 'Sweet Tooth, '  
    if (row['Escape or Coping Mechanism'] == 1.0):
        perc_cohort = perc_cohort + 'Coping Mechanism, '          
    if (row['Overweight Struggling to Lose'] == 1.0):
        perc_cohort = perc_cohort + 'Cant lose weight, '   
    if (row['Smoker'] == 1.0):
        perc_cohort = perc_cohort + 'Smoker, '           
    if (row['Failed Attempts to Stop Smoking'] == 1.0):
        perc_cohort = perc_cohort + 'Cant Quit Smoking, '         
    if (row['Binge Drinking'] == 1.0):
        perc_cohort = perc_cohort + 'Binge Drinking, '       
    if (row['Sedentary Lifestyle'] == 1.0):
        perc_cohort = perc_cohort + 'Sedentary Lifestyle, '        
    if (row['Self Efficacy'] == 1.0):
        perc_cohort = perc_cohort + 'Self Efficacy, '   
    if (row['Poor Social Support'] == 1.0):
        perc_cohort = perc_cohort + 'Poor Social Support, '          
 
        
    row['Poor Emotional Regulatory Control Cohort'] = perc_cohort
    return row    
              
              

In [8]:
# calculate all risk factors 

df_PERC=df_PERC.apply(interpret_acute_childhood_event, axis='columns')
df_PERC=df_PERC.apply(interpret_anxiety_depression_history, axis='columns')
df_PERC=df_PERC.apply(interpret_financial_stress, axis='columns')
df_PERC=df_PERC.apply(interpret_binging_overeating, axis='columns')
df_PERC=df_PERC.apply(interpret_anxiety_depression_history, axis='columns')
df_PERC=df_PERC.apply(interpret_sweet_tooth, axis='columns')
df_PERC=df_PERC.apply(interpret_overweight_struggling_to_lose, axis='columns')
df_PERC=df_PERC.apply(interpret_anxiety_depression_history, axis='columns')
df_PERC=df_PERC.apply(interpret_binge_drinking, axis='columns')
df_PERC=df_PERC.apply(interpret_smoking_risk, axis='columns')
df_PERC=df_PERC.apply(interpret_stop_smoking_risk, axis='columns')
df_PERC=df_PERC.apply(interpret_sedentary, axis='columns')
df_PERC=df_PERC.apply(interpret_self_efficacy, axis='columns')
df_PERC=df_PERC.apply(interpret_social_support_network, axis='columns')
df_PERC=df_PERC.apply(interpret_escape_or_coping_mechanism, axis='columns')

 
df_PERC=df_PERC.apply(sum_PERC_risk_Factors, axis='columns')
df_PERC=df_PERC.apply(sum_PERC_risk_Strat, axis='columns')
df_PERC=df_PERC.apply(sum_PERC_risk_Score, axis='columns')
df_PERC=df_PERC.apply(sum_PERC_risk_Cohort, axis='columns')

In [8]:
df_PERC['Poor Emotional Regulatory Control Factors'].value_counts() 

0     4435
4     2662
3      603
2      564
7      487
1      422
6      362
5      264
8      109
9       73
10      16
11       3
Name: Poor Emotional Regulatory Control Factors, dtype: int64

In [9]:
df_PERC['Poor Emotional Regulatory Control Cohort'].value_counts() 

                                                                                                                                                              485
Binge Eating, Sweet Tooth, Cant lose weight, Sedentary Lifestyle,                                                                                             283
Financial Stress, Self Efficacy, Poor Social Support,                                                                                                          53
Cant Quit Smoking,                                                                                                                                             37
Financial Stress, Binge Eating, Sweet Tooth, Cant lose weight, Sedentary Lifestyle, Self Efficacy, Poor Social Support,                                        35
Coping Mechanism, Binge Drinking,                                                                                                                              25
Binge Eating, Sweet Tooth, C

In [10]:
df_PERC.head(10)


,Age,Gender,Diabetes,BMI,Simulated BMI Class,Total Calculated AIHS Risk Stratifcation,Do you have one person you think of as your personal doctor or health care provider?,History of Anxiety and Depression,Alcohol Days last 30 Days or Week,During the past 30 days on the days when you drank about how many drinks average,...,Smoker,Failed Attempts to Stop Smoking,Sedentary Lifestyle,Self Efficacy,Poor Social Support,Escape or Coping Mechanism,Poor Emotional Regulatory Control Factors,Poor Emotional Regulatory Control Risk,Poor Emotional Regulatory Control Score,Poor Emotional Regulatory Control Cohort
0,49,Male,0,40.175405,Obese,6.70,1,NaN,888.0,NaN,...,0,0,1,0,0,0,4,0.285714,4.0,"Binge Eating, Sweet Tooth, Cant lose weight, S..."
1,73,Male,0,25.087907,Healthy Weight,5.30,1,NaN,888.0,NaN,...,0,0,0,1,1,0,3,0.214286,3.0,"Financial Stress, Self Efficacy, Poor Social S..."
2,49,Male,0,22.036301,Over Weight,5.01,2,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0.000000,0.0,
3,87,Female,0,28.191696,Obese,9.55,2,NaN,888.0,NaN,...,0,0,0,1,1,0,3,0.214286,3.0,"Financial Stress, Self Efficacy, Poor Social S..."
4,87,Male,0,24.373994,Obese,10.25,1,NaN,888.0,NaN,...,0,0,0,0,0,0,0,0.000000,0.0,
5,59,Male,0,26.520575,Obese,6.36,1,NaN,888.0,NaN,...,0,0,0,0,0,0,0,0.000000,0.0,
6,96,Female,0,23.887580,Over Weight,3.08,1,NaN,203.0,1.0,...,0,0,0,0,0,0,0,0.000000,0.0,
7,91,Male,0,33.821407,Over Weight,1.30,1,NaN,888.0,NaN,...,0,0,1,1,1,0,7,0.500000,7.0,"Financial Stress, Binge Eating, Sweet Tooth, C..."
8,74,Female,0,14.879779,Obese,9.16,1,NaN,888.0,NaN,...,0,0,0,0,0,0,0,0.000000,0.0,
9,68,Male,0,25.215906,Healthy Weight,3.80,1,NaN,888.0,NaN,...,0,0,0,0,0,0,0,0.000000,0.0,


In [11]:
df_PERC.rename(columns={"ADANXEV": "Anxiety and Depression"}, inplace=True)
df_PERC.rename(columns={"ALCDAY5": "Alcohol Days last 30 Days or Week"}, inplace=True)
df_PERC.rename(columns={"SMOKE100": "have you smoked >100 cigerettes in life"}, inplace=True)
df_PERC.rename(columns={"SMOKDAY2": "Do you now smoke cigarettes every day, some days, or not at all?"}, inplace=True)
df_PERC.rename(columns={"USENOW3": "Do you currently use any tobacco products?"}, inplace=True)
df_PERC.rename(columns={"STOPSMK2": "Have you tried to stop smoking?"}, inplace=True)
df_PERC.rename(columns={"MEDCOST": "Have you avioided necessary medical do to cost in last year?"}, inplace=True)
df_PERC.rename(columns={"ALCDAY5": "During the past 30 days how many days per week or per month did you have at least one drink of any alcoholic"}, inplace=True)
df_PERC.rename(columns={"AVEDRNK2": "During the past 30 days on the days when you drank about how many drinks average"}, inplace=True)
df_PERC.rename(columns={"MAXDRNKS": "During the past 30 days, what is the largest number of drinks you had on any occasion?"}, inplace=True)
df_PERC.rename(columns={"DRNK3GE5": "Any alcoholic beverages, how many times during the past 30 days did you have"}, inplace=True)


In [12]:
# Interpret binge Drinking
df_PERC=df_PERC.apply(interpret_binge_drinking, axis='columns')


In [13]:
df_PERC['Binge Drinking'].value_counts() 

0    935
1     65
Name: Binge Drinking, dtype: int64

In [14]:
df_PERC.to_csv("C:/Users/josep/documents/datasets/patient_PERC_factors.csv")

# Poor Emotional Regulation Control 

In [15]:
1. Poor Emotional Regulation Cohort (ORANGE)
1.	High ACE score (>5 out of 10)
2.	History of anxiety and/or depression (but exclude current depression)
3.	More likely to be low income or struggling financially
4.	Binging, overeating, emotional eating
5.	Sweet tooth, eats dessert, has cravings
6.	Likely overweight and perpetually struggling to lose weight
7.	Regular alcohol consumption, TV watching or other escape mechanisms
8.	Can't manage to quit smoking
9.	Probably sedentary or low exercise
10.	Low self efficacy/confidence (due to repeated failure with diets and exercise programs)
11.	Lack of social support network (few friends, loneliness)
12.	EXCLUDE: depression, drug use


SyntaxError: invalid syntax (4151274048.py, line 1)

In [19]:
from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *

# Widgets
widget1 = Dropdown(options = [1,2,3],description = 'Login Patient')
patientindex = display(widget1)

Dropdown(description='Login Patient', options=(1, 2, 3), value=1)

In [20]:
print("enter your login name")
name=input()

enter your login name
Joe Eberle


In [26]:
from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *

# Widgets
widget2 = Dropdown(options = ['Demographic','Simulated BMI','Nutrition'],description = 'What data?')
topic = display(widget2)


Dropdown(description='What data?', options=('Demographic', 'Simulated BMI', 'Nutrition'), value='Demographic')

In [27]:

print(topic.value())

AttributeError: 'NoneType' object has no attribute 'value'

In [23]:
if topic.value() >= 'Demographic':
    df_demo = pd.DataFrame(df_Patient.loc[1:2 , ['Age', 'Gender', 'Race']]) 
else:
    df_demo = pd.DataFrame(df_Patient.loc[1:2 , ['Age', 'Race']]) 

df_demo.iloc[1:2 , : ]


AttributeError: 'NoneType' object has no attribute 'value'

In [28]:
type(topic) 

NoneType

In [29]:
print("enter your login name")
name=input()

questions_list = ["Q1. Body, Mind, or Soul ?","Q2. Want to see the data II have about you ? ", "Q3. What do you want help with? ?"]

answers_list = [["A.Body ", " B.Mind ", " C.Soul "],
               ["A.Demographic ", " B.BMI ", " C.Nutrition "," D.Pre-Diabetes Risk"],
               ["A.Increasing Acitivity", " B.Losing Weight ", " C.Nutrition "," D.Managing Your Diabetes Risk"],]

for questions, answers in zip(questions_list, answers_list):
    print(questions + '\n' + ''.join(answers)+'\n')
    get_answer = input()
    print('\n')


enter your login name
j
Q1. Body, Mind, or Soul ?
A.Body  B.Mind  C.Soul 

a


Q2. Want to see the data II have about you ? 
A.Demographic  B.BMI  C.Nutrition  D.Pre-Diabetes Risk

b


Q3. What do you want help with? ?
A.Increasing Acitivity B.Losing Weight  C.Nutrition  D.Managing Your Diabetes Risk

a




In [31]:
print("enter your login name")
name=input()

questions_list = ["Q1. Body, Mind, or Soul ?","Q2. Want to see the data II have about you ? ", "Q3. What do you want help with? ?"]

answers_list = [["A.Body ", " B.Mind ", " C.Soul "],
               ["A.Demographic ", " B.BMI ", " C.Nutrition "," D.Pre-Diabetes Risk"],
               ["A.Increasing Acitivity", " B.Losing Weight ", " C.Nutrition "," D.Managing Your Diabetes Risk"],]


for questions, answers in zip(questions_list, answers_list):
    print(questions + '\n' + ''.join(answers)+'\n')
    get_answer = input()
    print('\n')
    
    
#     1
# Strongly disagree	2
# Disagree	3
# Neither Agree nor disagree	4
# Agree	5
# Strongly agree

enter your login name
j
Q1. Body, Mind, or Soul ?
A.Body  B.Mind  C.Soul 

a


Q2. Want to see the data II have about you ? 
A.Demographic  B.BMI  C.Nutrition  D.Pre-Diabetes Risk

b


Q3. What do you want help with? ?
A.Increasing Acitivity B.Losing Weight  C.Nutrition  D.Managing Your Diabetes Risk

a




In [ ]:
print("enter your login name")
name=input()

In [ ]:

bmi = df_Patient['BMI Class'].value_counts() 
# Returns object containing counts of unique values.
# The resulting object will be in descending order so that the first element is the most frequently-occurring element. 
# Excludes NA values by default.
print(type(bmi))
bmi.head()

people = df_Patient['Gender'].value_counts() 
# Returns object containing counts of unique values.
# The resulting object will be in descending order so that the first element is the most frequently-occurring element. 
# Excludes NA values by default.
print(type(people))
people.head()


fig = plt.figure(figsize=(20,8))
plt.title("Total Data",loc='center',weight=10,size=15)
plt.xticks([]) # to disable xticks
plt.yticks([]) # to disable yticks

# first pie-plot
ax1 = fig.add_subplot(121)
ax1.axis('equal')
explode = (0.01,)*(len(people))

wedges, texts, autotexts =  ax1.pie(people,
                                    radius=0.8,
                                    explode=explode,
                                    labels=['female','male'],
                                    colors=['#f7879a','#49759c'],
                                    autopct="%1.1f%%",
                                    pctdistance=0.7,
                                    textprops=dict(color='k'),
                                    wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'w' }
                                    )
plt.setp(autotexts,size=17)
plt.setp(texts,size=15)
my_circle = plt.Circle((0,0),0.5,color='white')
p = plt.gcf() # get current figure reference
p.gca().add_artist(my_circle) # get current axes

# Second pie-plot
ax2 = fig.add_subplot(122)
ax2.axis('equal')
explode = (0.01,)*(len(bmi))
wedges2, texts2, autotexts2 =  ax2.pie(bmi,
                                    radius=0.8,
                                    explode=explode,
                                    labels=['Overweight','Obese','Healthy Weight','Morbidly Obese','Underweight'],
                                    autopct="%1.1f%%",
                                    pctdistance=0.5,
                                    textprops=dict(color='k'),
                                    wedgeprops = { 'linewidth' : 3, 'edgecolor' : 'w' }
                                    )
plt.setp(autotexts2,size=17)
plt.setp(texts2,size=15)
my_circle = plt.Circle((0,0),0.5,color='white')
p = plt.gcf() # get current figure referenceq
p.gca().add_artist(my_circle) # get current axes

In [ ]:
pd.DataFrame(df_Patient.Gender.value_counts())
pd.DataFrame(df_Patient.Race.value_counts())

In [ ]:
cols = df_Patient.columns
df_Patient.shape
cols


In [ ]:
df_Patient_Demographic = df_Patient[[ 'Age', 'Decade','Gender', 'Race', 'Race Category','BMI','Activity Level','Physical Activity Minutes','Hypertension','Diabetes']]
df_Patient_Demographic

In [ ]:
df_Patient_Demographic.BMI.describe

In [ ]:
pd.DataFrame(df_Patient_Demographic.Gender.value_counts())


In [ ]:
correlation_matrix = df_Patient_Demographic.corr()     #establish a correlation matrix for all fields
top_correlation_features = correlation_matrix.index
plt.figure(figsize=(20,20))
g=sns.heatmap(df_Patient_Demographic[top_correlation_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
df_BRFSS_REDUCED['Gender'].value_counts() 

In [ ]:
# Read in the BRFSS data set 
df_BRFSS = pd.read_csv("C:/Data/brfss cdc/BRFSS_2015.csv",nrows=10000)
df_BRFSS.shape

In [ ]:
# Save the transformed file 
df_BRFSS.to_csv("C:/Users/josep/documents/datasets/BRFSS_Redux.csv") 

In [ ]:
# Read in the integrated Data Files 
df_BRFSS_RAW = pd.read_csv("C:/Users/josep/documents/datasets/AIHS_SIM_V4_BRFSS_RAW.csv") 
df_BRFSS_RAW.head()

In [ ]:
# Save the transformed file 
df_BRFSS.to_csv("C:/Users/josep/documents/datasets/AIHS_BRFSS_RAW.csv") 

In [ ]:
df_BRFSS.head()

In [ ]:
# Calculate the AIHS health risk stratification 
def calc_age_risk(age):
    if age < 40: 
        age_risk =  0.0
    elif age < 50: 
        age_risk =  1 + ((age-40) * .1)
    elif age  < 60: 
        age_risk =  2 + ((age-50) * .1)
    elif age  < 90: 
        age_risk =  3 + ((age-60) * .1)
    elif age  < 110:    # Age over 90 non actionable ????
        age_risk =  0      
    
    return age_risk   

In [ ]:
kilo_grams = float(input('Enter weight in Kg to Convert into pounds:'))
pounds = kilo_grams * 2.2046
print(kilo_grams,' Kilograms =', pounds,' Pounds')

In [ ]:
def calc_AIHS_age_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    # Let's extract the firstname and create a new entry in the series
    if row['Age'] < 40: 
        row['AIHS Age Risk']=  0.0
    elif row['Age'] < 50: 
        row['AIHS Age Risk']=  1 + ((row['Age']-40) * .1)
    elif row['Age']  < 60: 
        row['AIHS Age Risk']=  2 + ((row['Age']-50) * .1)
    elif row['Age']  < 90: 
        row['AIHS Age Risk']=  3 + ((row['Age']-60) * .1)
    elif row['Age']  < 110:    # Age over 90 non actionable ????
        row['AIHS Age Risk']=  0      
   
    return row

In [ ]:
def calc_height_in_feet(row):
    # Converts from centimeters to feet and inches 
    row['Simulated Height IN']=  round(row['Simulated Height CM']  * 0.0328,1) 
    return row

In [ ]:
def calc_weight_in_pounds(row):
    # Converts from centimeters to feet and inches 
    row['Simulated Weight LBS']=   round(row['Simulated Weight KG']  * 2.2046,1)
    return row

In [ ]:
def calc_body_mass_index(row):
    # Converts from centimeters to feet and inches 
    row['Simulated BMI']=  row['Simulated Weight KG']/(row['Simulated Height CM']**2)  
    return row

In [ ]:
def calc_ADA_age_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    # Let's extract the firstname and create a new entry in the series
    if row['Age'] < 40: 
        row['ADA Age Risk']= 0
    elif row['Age'] < 50: 
        row['ADA Age Risk']=  1  
    elif row['Age']  < 60: 
        row['ADA Age Risk']=  2  
    elif row['Age']  < 90: 
        row['ADA Age Risk']=  3  
    elif row['Age']  < 110:    # Age over 90 non actionable ????
        row['ADA Age Risk']=  0      
    return row

In [ ]:
def calc_ADA_BMI_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    # Let's extract the firstname and create a new entry in the series
    if row['Simulated BMI'] < 18.5: 
        row['ADA BMI Risk']= 0
    elif row['Simulated BMI'] < 25: 
        row['ADA BMI Risk']=  0  
    elif row['Simulated BMI']  < 30: 
        row['ADA BMI Risk']=  1  
    elif row['Simulated BMI']  < 40: 
        row['ADA BMI Risk']=  2  
    elif row['Simulated BMI']  < 60:    # Age over 90 non actionable ????
        row['ADA BMI Risk']=  3      
    return row

In [ ]:
def calc_AIHS_BMI_risk(row):
    # The row is a single Series object which is a single row indexed by column values
    # Let's extract the firstname and create a new entry in the series
    if row['Simulated BMI'] < 18.5: 
        row['AIHS BMI Risk']= 0
    elif row['Simulated BMI'] < 25: 
        row['AIHS BMI Risk']=  0  
    elif row['Simulated BMI']  < 30: 
        row['AIHS BMI Risk']=  1 +  ((row['Simulated BMI']-26) * .1)
    elif row['Simulated BMI']  < 40: 
        row['AIHS BMI Risk']=  2 +  ((row['Simulated BMI']-31) * .1) 
    elif row['Simulated BMI']  < 60:    # Age over 90 non actionable ????
        row['AIHS BMI Risk']=  3 +  ((row['Simulated BMI']-41) * .1)      
    return row

## Read In the core demographic data set 

In [ ]:
#df = pd.read_csv("C:/Users/josep/documents/datasets/dg.csv", nrows=5000)
df = pd.read_csv("C:/Users/josep/documents/datasets/dg_BMI_risk.csv")
df.head(5)

In [ ]:
df.head()

In [ ]:
cols = df.columns
print(cols)

In [ ]:
df['Simulated Height CM'] = np.random.randint(155, 187, df.shape[0])
df['Simulated Weight KG'] = np.random.randint(60, 130, df.shape[0])
df['Simulated BMI']=  round(df['Simulated Weight KG']/((df['Simulated Height CM']/100)**2),1)  
df.head() 

In [ ]:
height = float(input("Enter height in meters: "))

weight = float(input("Enter weight in kg: "))

bmi = weight/(height**2)

print("Your BMI is: {0} and you are: ".format(bmi), end='')

if ( bmi < 16):
    print("severely underweight")

elif ( bmi >= 16 and bmi < 18.5):
    print("underweight")

elif ( bmi >= 18.5 and bmi < 25):
    print("Healthy")

elif ( bmi >= 25 and bmi < 30):
    print("overweight")

elif ( bmi >=30):
    print("severely overweight")

In [ ]:
df.to_csv("C:/Users/josep/documents/datasets/dg_BMI_risk_III.csv")

In [ ]:
df_BRFSS = pd.read_csv("C:/Data/brfss cdc/BRFSS_2015.csv",nrows=5000)
df_BRFSS.shape

In [ ]:
df_BRFSS["Patient Number"]= np.arange(len(df_BRFSS)) + 1

In [ ]:
df_BRFSS["Patient ID"]= "Patient_Number_" + df_BRFSS["Patient Number"].astype(str)

In [ ]:
df_BRFSS.head()

In [ ]:
df["Age Decade"]=df['Age'] / 10 
df["Patient Number"]= np.arange(len(df)) + 1 
cols = df.columns
df.head()

In [ ]:
df["Age Dec"]=df['Age Decade'].astype(int) 
df["Patient ID"]= "Patient_Number_" + df["Patient Number"].astype(str)
cols = df.columns
df.head()

In [ ]:
dfm = df.merge(df_BRFSS, how='left', on='Patient ID')

In [ ]:
dfm.shape

In [ ]:
#take input from user
cm=int(input("Enter the height in centimeters:"))
 
#convert centimeter to feet
feet=0.0328*cm
 
#print result
print("The length in feet",round(feet,2))


In [ ]:
dfm.head()

In [ ]:
df.head()

In [ ]:
dfm.to_csv("C:/Users/josep/documents/datasets/patient_demo_behavior.csv")

In [ ]:
df["Age Dec"]=df['Age Decade'].astype(int) 
df["Patient ID"]= "Patient_Number_" + df["Patient Number"].astype(str)
cols = df.columns
df.head()

In [ ]:
#df=df.apply(calc_ADS_age_risk, axis='columns')
df=df.apply(calc_height_in_feet, axis='columns')
df.head()

In [ ]:
#df=df.apply(calc_ADS_age_risk, axis='columns')
df=df.apply(calc_weight_in_pounds, axis='columns')
df.head()

In [ ]:
#cols = cols + 'Computed AIHS Age Risk'
df = df[cols]
df.head(10)



In [ ]:
#df=df.apply(calc_ADS_age_risk, axis='columns')
df=df.apply(calc_AIHS_age_risk, axis='columns')
df.head()


In [ ]:
#df=df.apply(calc_ADS_age_risk, axis='columns')
df=df.apply(calc_ADA_BMI_risk, axis='columns')
df.head()

In [ ]:
#df=df.apply(calc_ADS_age_risk, axis='columns')
df=df.apply(calc_AIHS_BMI_risk, axis='columns')
pd.options.display.max_columns = None
df.head(5)